## 例3-1. nグラムの計算

In [2]:
import pandas as pd
import json
from sklearn.feature_extraction.text import CountVectorizer

# 最初の 10,000 件のレビューを読み込む
with open('data/yelp_academic_dataset_review.json') as f:
    js = []
    for i in range(10000):
        js.append(json.loads(f.readline()))
review_df = pd.DataFrame(js)

# scikit-learn の CountVectorizer を使ってユニグラム（BoW）、
# バイグラム、トライグラムの特徴量変換器を作成する。
# CountVectorizer はデフォルトでは1文字の単語を無視するが、
# これは意味のない単語を除外するため実用的である。
# ただしここでは全ての単語を含むように設定している。
bow_converter = CountVectorizer(token_pattern='(?u)\\b\\w+\\b')
bigram_converter = CountVectorizer(ngram_range=(2,2), token_pattern='(?u)\\b\\w+\\b')
trigram_converter = CountVectorizer(ngram_range=(3,3), token_pattern='(?u)\\b\\w+\\b')

# 変換器を適用し、語彙数を確認する
bow_converter.fit(review_df['text'])
words = bow_converter.get_feature_names_out()

bigram_converter.fit(review_df['text'])
bigrams = bigram_converter.get_feature_names_out()

trigram_converter.fit(review_df['text'])
trigrams = trigram_converter.get_feature_names_out()

print (len(words), len(bigrams), len(trigrams))

# n-グラムを確認する
words[:10]

29222 368943 881620


array(['0', '00', '000', '007', '00a', '00am', '00pm', '01', '02', '03'],
      dtype=object)

In [12]:
bigrams[-10:]

array(['zuzu was', 'zuzus room', 'zweigel wine', 'zwiebel kräuter',
       'zy world', 'zzed in', 'éclairs napoleons', 'école lenôtre',
       'ém all', 'òc châm'], dtype=object)

In [6]:
trigrams[:10]

array(['0 0 eye', '0 20 less', '0 39 oz', '0 39 pizza', '0 5 i',
       '0 50 to', '0 6 can', '0 75 oysters', '0 75 that', '0 75 to'],
      dtype=object)

## 例3-2. 品詞タグ付けとチャンク化

In [14]:
! pip install spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 9.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.7/128.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 924.9/924.9 kB 9.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.6/491.6 kB 10.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 10.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 9.6 MB/s eta 0:00:00:00:010:01


In [20]:
! python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 8.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [21]:
import pandas as pd
import json

# 最初の10レビューを読み込む
with open('data/yelp_academic_dataset_review.json') as f:
    js = []
    for i in range(10):
        js.append(json.loads(f.readline()))
review_df = pd.DataFrame(js)

# まずは Spacy を使った方法
import spacy

# 言語モデル（英語）を読み込む
#nlp = spacy.load('en')
nlp = spacy.load("en_core_web_sm")
import en_core_web_sm

# spaCy の言語モデルを使ってテキストから Pandas Series を作成する
doc_df = review_df['text'].apply(nlp)

# spaCy は細かい品詞タグを .pos_ で、粗い品詞タグを .tag_ で提供します
for doc in doc_df[4]:
    print([doc.text, doc.pos_, doc.tag_])

# spaCy は基本的な名詞句も .noun_chunks で提供します
print([chunk for chunk in doc_df[4].noun_chunks])

['General', 'PROPN', 'NNP']
['Manager', 'PROPN', 'NNP']
['Scott', 'PROPN', 'NNP']
['Petello', 'PROPN', 'NNP']
['is', 'AUX', 'VBZ']
['a', 'DET', 'DT']
['good', 'ADJ', 'JJ']
['egg', 'NOUN', 'NN']
['!', 'PUNCT', '.']
['!', 'PUNCT', '.']
['!', 'PUNCT', '.']
['Not', 'PART', 'RB']
['to', 'PART', 'TO']
['go', 'VERB', 'VB']
['into', 'ADP', 'IN']
['detail', 'NOUN', 'NN']
[',', 'PUNCT', ',']
['but', 'CCONJ', 'CC']
['let', 'VERB', 'VB']
['me', 'PRON', 'PRP']
['assure', 'VERB', 'VB']
['you', 'PRON', 'PRP']
['if', 'SCONJ', 'IN']
['you', 'PRON', 'PRP']
['have', 'VERB', 'VBP']
['any', 'DET', 'DT']
['issues', 'NOUN', 'NNS']
['(', 'PUNCT', '-LRB-']
['albeit', 'ADV', 'RB']
['rare', 'ADJ', 'JJ']
[')', 'PUNCT', '-RRB-']
['speak', 'VERB', 'VBP']
['with', 'ADP', 'IN']
['Scott', 'PROPN', 'NNP']
['and', 'CCONJ', 'CC']
['treat', 'VERB', 'VB']
['the', 'DET', 'DT']
['guy', 'NOUN', 'NN']
['with', 'ADP', 'IN']
['some', 'DET', 'DT']
['respect', 'NOUN', 'NN']
['as', 'SCONJ', 'IN']
['you', 'PRON', 'PRP']
['state', 'V